<a href="https://colab.research.google.com/github/bagcheap/oreilly_live_training_agents/blob/main/notebooks/1.0-intro-agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install openai

# Why Agents?

What is an agent?

An agent is a reference to combining models that can perform some kind of reasoning, like large language models (e.g ChatGPT, Llama2, Mistral, etc...) with tools to give it access to the real world,
so they can do things like browsing the internet, buying stuff, etc...

Ok, so, why is there so much hype around agents right now?

Because Agents are cool! Recently with the advance of LLMs, we've seen them become an amazing tool to do all sorts of things like building apps, browse the internet and more.



SOme neat examples of these kinds of agents can be found in here:

- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)

Now, today although they seem extremely powerful, agents are still at a very early stage in terms of readiness to deploy as products, something you can atest by listening to Andrej Karpathy talk about agents in this talk here:

- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)

This live-training is all about! Getting you excited about this amazing new technology, understanding it from the ground up but with a focus on practical applications and fun stuff you can do with them!

# What is an Agent?

An agent is nothing more than some entity that can _think_ and _act_, that's right, in a way you're an agent!

After all you can think and act on those thoughts like in the case of coming to this live-training:

- Thought: "I want to learn about agents"

- Action: "Go to the internet and research cool platforms where I can learn about agents"

- Thought: "O'Reilly has some awesome courses and live-trainings"

- Action: "Look up O'Reilly courses"

- Thought: "Live-trainings by instructor Lucas are awesome"

- Action: "Schedule live-training about agents with instructor Lucas Soares" (lol)

In a way this is a simplified rendition of what brought you here, obviously not necessarily in this particular order nor these particular sets of thought and action pairs. This particular way of thinking about how to structure thoughts and actions is well represented in the paper: [ReACT](https://arxiv.org/pdf/2210.03629.pdf).

With regards to LLMs, how can bring this idea to fruition thinking about the LLM model as the reasoning and thinking engine?

We can start simple and just call the openai API to start:

In [3]:
# uncomment this if running locally
#!pip install python-dotenv
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [4]:
from google.colab import userdata
# Retrieve the API key from Colab Secrets
api_key = userdata.get('OPENAI_API_KEY')

In [5]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI(api_key=api_key)

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )

    return response.choices[0].message.content

output = get_response("Create a joke about a dog and a cat")
Markdown(output)

Why did the cat bring a calculator to the dog park?

Because he wanted to paw-sitively calculate how many tennis balls the dog would try to steal!

Ok cool, so here we have three ideas of actions to perform:

- Creating directories
- Listing files
- Removing files

Let's transform them into functions that we could call just like in any type of Python-based application.

In [6]:
# NOTE: I changed the functions in these examples from the slides (which use subprocess), but they do the same thing
# I am just using the os module here so people can test this on google colab

import os

def create_directory():
    os.makedirs('test', exist_ok=True)

def create_file():
    with open('test.txt', 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

Notice that here we are not concerned with giving these functions arguments to avoid complicating things, the only thing we should focus on is to have the ability to perform the actions outside the narrow scope of the python script out there in the real-world.

Let's test these functions

In [7]:
!ls

sample_data


Ok, we start with this folder containing a few notebooks, scripts and sub-directories.

Let's test each function and inspect its output behavior:

In [8]:
create_directory()

!ls

sample_data  test


In [9]:
create_file()

!ls

sample_data  test  test.txt


In [10]:
list_files()

.config
test.txt
test
sample_data


Ok great all the functions are working!

Now, let's imagine that we wanted to create an agent that would perform these actions for us based on some input that we give it, how can we connect models that we know and can use today like ChatGPT, with these tools that do stuff in the real world?

To answer this question, how about we give a task to the model, and for that task we ask it to list the steps that it needs to perform to complete the task, and then for each of those steps we would ask the model to decide whether or not a function should be called to execute that task?

This is what the now famous paper ['Toolformer'](https://arxiv.org/pdf/2302.04761.pdf) demonstrated!

They showed that today's advanced LLMs like the gpt-series could teacha themselves how to properly call and use external tools!

Isn't that awesome???

So, let's see if we can hack our way into connecting the llm response with the functions that we want that llm to use.

In [10]:
def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )

    return response.choices[0].message.content


def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

OK, cool! now Notice that, here we added single parameters to the functions: `create_directory(), create_file()`, and we did this so
that we can actually do real things instead of just always creating the same folders over and over.

Now, how can we actually put it all together so that given a task, a model can:

- Plan the task
- Execute actions to complete the task
- Know when to call a function

????

This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [11]:
def get_response(prompt_question, model="gpt-3.5-turbo-16k"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )

    return response.choices[0].message.content

def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)



task_description = "Create a folder called 'angus-the-agent-master'. Inside that folder, create a file called 'the-10-master-rules.md"
output = get_response(f"""Given this task: {task_description}, \n
                            Consider you have access to the following functions:

    def create_directory(dir_name):
        os.makedirs(dir_name, exist_ok=True)

    def create_file():
        with open('test.txt', 'w'):
            pass

    def list_files():
        files = os.listdir()
        for file in files:
            print(file)

    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """)

Markdown(output)

create_directory('angus-the-agent-master')

Hey! Look at that the output is that function! Now, all we need is find a way to execute this function. We can use Python's built in `exec` method for that:

In [8]:
output

'create_directory("angus-the-agent-master")'

In [9]:
exec(output)

In [ ]:
!ls -d */ | grep lucas

lucas-agent-007-the-return-of-pancake-man/
lucas-agent-007/
lucas-loves-pancakes/
lucas-still-loves-pancakes/
lucas-test/
lucas-the-agent-master/


Yessss! We did it! All we had to do is to use the Python builtin method `exec` connected with the function call we got from the model's response!

This is great, but what if we wanted to perform multiple actions?

How about changing our prompt so that our output is a python list of function calls which we can later programmatically call?

Let's try that:

In [ ]:
task_description = "Create a folder called 'lucas-the-agent-master'. Inside that folder create a file called 'the-10-master-rules.md'."
output = get_response(f"""Given a task that will be fed as input, and consider you have access to the following functions:

    def create_directory(dir_name):
        os.makedirs(dir_name, exist_ok=True)

    def create_file(filename):
        with open(filename, 'w'):
            pass

    def list_files():
        files = os.listdir()
        for file in files:
            print(file)
    .
    Your output should be the a list of function calls to be executed to complete the task containing the necessary arguments.
    For example:

    task: 'create a folder named test-dir'
    output_list: [create_directory('test-dir')]

    task: 'create a file named file.txt'
    output_list: [create_file('file.txt')]

    task: 'Create a folder named lucas-dir and inside that folder create a file named lucas-file.txt'
    output_list: [create_directory('lucas-dir'), create_file('lucas-dir/lucas-file.txt')]

    The OUTPUT SHOULD ONLY BE A PYTHON LIST WITH THE FUNCTION CALLS INSIDE and NOTHING ELSE.
    task: {task_description}
    output_list:\n
    """)

Markdown(output)

[create_directory('lucas-the-agent-master'), create_file('lucas-the-agent-master/the-10-master-rules.md')]

In [ ]:
exec(output)

In [ ]:
!ls lucas-the-agent-master/

the-10-master-rules 2.md the-10-master-rules.md


At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)